---

## Step 0: Prepare image repository

### Generate images from pixels

We begin with generating images from pixel dataset and storing them in their respective folders according thier emotion class and category (Train, PrivateTest, PublicTest). 
The library `PIL` is used to convert pixels to images. 
- `fer2013.csv` - base dataset hosted by the Kaggle competition

Each row represents a face. There are 3 columns:
- `emotions` - emotion expressed by the face
- `pixels` - this column contains 2304 pixel values in space-separated manner
- `category` - purpose of the image (Train, PrivateTest, PublicTest)


In [ ]:
import numpy as np
import scipy.misc as smp
import csv
from PIL import Image
import pandas as pd
count = 1
strEmotion = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

In [ ]:
with open('fer/fer2013.csv', 'rt') as csvfile:
    spamreader = csv.reader(csvfile)
    for row in spamreader:
        x = row[1].split()
        x1=np.reshape(x,(48,48))
        img = Image.new( 'RGB', (48,48), "black") 
        pixels = img.load() # create the pixel map
        for i in range(img.size[0]):    # for every col:
            for j in range(img.size[1]):    # For every row
                pixels[i,j] = (int(x1[j][i]), 0, 0) 

        img.save('fer/'+row[2]+'/'+row[0]+'_'+strEmotion[int(row[0])]+'/'+row[0] +'_'+str(count)+'.jpg')
        count += 1


## Step 1: Import image data

### Read images as 4D tensors


In [ ]:


# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    emo_files = np.array(data['filenames'])
    emo_targets = np_utils.to_categorical(np.array(data['target']), 7)
    return emo_files, emo_targets

# load train, test, and validation datasets
train_files, train_targets = load_dataset('fer/Training')
valid_files, valid_targets = load_dataset('fer/PrivateTest')
test_files, test_targets = load_dataset('fer/PublicTest')

# load list of emotions
emo_names = [item[13:-1] for item in sorted(glob("fer/Training/*/"))]

# print statistics about the dataset
print('There are %d total emotion categories.' % len(emo_names))
print('There are %s total facial images.\n' % len(np.hstack([train_files, valid_files, test_files])))
print('There are %d training facial images.' % len(train_files))
print('There are %d validation facial images.' % len(valid_files))
print('There are %d test facial images.'% len(test_files))

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    img = image.load_img(img_path,grayscale=True, target_size=(48, 48))
    x = image.img_to_array(img)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
img_path = 'test_images/6_32609.jpg'
zz = path_to_tensor(img_path)
shape()

---
<a id='step3'></a>
## Step 2: Rescale the image 

### Pre-process the Data

We rescale the images by dividing every pixel in every image by 255.

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

train_tensors = paths_to_tensor(train_files).astype('float32')/255
valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
import tensorflow as tf 
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.merge_ops import merge_outputs, merge
from tflearn.layers.normalization import local_response_normalization, batch_normalization
from tflearn.layers.estimator import regression 
from tflearn.optimizers import Momentum, Adam


## Step 3: Model Architecture

The code cell below defines the model architecture: 

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential
from keras.initializers import TruncatedNormal,glorot_normal
import math

model = Sequential()

n = 0.01
model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu',input_shape=(48, 48, 1)
                 ,kernel_initializer=glorot_normal(seed=0)
                 ,bias_initializer='zeros',name = "Input"))

model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=3, strides=2))

n = 0.01
model.add(Conv2D(filters=32, kernel_size=5, strides=1, padding='same', activation='relu'
                 ,kernel_initializer=glorot_normal(seed=0)
                 ,bias_initializer='zeros'))

model.add(Dropout(0.4))
model.add(MaxPooling2D(pool_size=3, strides=2))

model.add(Conv2D(filters=64, kernel_size=4, strides=1, padding='same', activation='relu'
                 ,kernel_initializer=glorot_normal(seed=0)
                 ,bias_initializer='zeros'))

model.add(Dropout(0.3))
#model.add(MaxPooling2D(pool_size=2, strides=2))


model.add(Flatten())

model.add(Dense(100,activation = 'relu'))
#model.add(Dense(15,activation = 'relu'))
#n = math.sqrt(2.0/(9*256))
model.add(Dense(7,activation='softmax',name="Output"))

model.summary()


### Compile the Model

In [ ]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])

### Train the Model

In [ ]:
from keras.callbacks import ModelCheckpoint,CSVLogger

epochs = 20
csv_logger = CSVLogger('logs/training.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.tudelft.hdf5', 
                               verbose=1, save_best_only=True)

hist = model.fit(train_tensors, train_targets, 
          validation_data=(valid_tensors, valid_targets),
          epochs=epochs, batch_size=50, callbacks=[checkpointer,csv_logger], verbose=0)

### Load the Model with the Best Validation Loss

In [ ]:
model.load_weights('saved_models/weights.best.tudelft.hdf5')
model.save('saved_models/model_tudelft.h5')

In [ ]:
from keras.models import load_model
model_saved = load_model('saved_models/model_tudelft.h5')

## Step 4: Test the Model

In [ ]:
emotion_predictions = [np.argmax(model_saved.predict(np.expand_dims(tensor, axis=0))) for tensor in test_tensors]

test_accuracy = 100*np.sum(np.array(emotion_predictions)==np.argmax(test_targets, axis=1))/len(emotion_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

## Step 5: Model prediction

In [ ]:
import cv2                
import matplotlib.pyplot as plt                        
%matplotlib inline
strEmotion = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']
def get_emotion(img_path):
    emotion_class = model_saved.predict_classes(path_to_tensor(img_path))
    return emotion_class

In [ ]:
def emotion_detector(img_path):
    emotion = get_emotion(img_path) 
    img = cv2.imread(img_path)
    cv_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(cv_rgb)
    plt.show()
    print("Emotion detected: " + str(strEmotion[np.asscalar(emotion)]))
    

In [ ]:
emotion_detector('test_images/6_32609.jpg')

In [ ]:
emotion_probs = model_saved.predict_classes(path_to_tensor('test_images/0_32611.jpg'))
emotion_probs

In [ ]:
from cv2 import __version__
__version__

In [ ]:
from keras.models import model_from_json
json_string = 'C:/Work/1. Coursera/4. MLND/machine-learning-master/machine-learning-master/projects/dog-project-master/to/tensorflow-101-master/model/facial_expression_model_structure.json'
#model = model_from_json(json_string)


model_file = open('C:/Work/1. Coursera/4. MLND/machine-learning-master/machine-learning-master/projects/dog-project-master/to/tensorflow-101-master/model/facial_expression_model_structure.json', 'r')
model = model_file.read()
model_file.close()
model_trans = model_from_json(model)
model_trans.load_weights("C:/Work/1. Coursera/4. MLND/machine-learning-master/machine-learning-master/projects/dog-project-master/to/tensorflow-101-master/model/facial_expression_model_weights.h5")
model_trans.summary()

In [ ]:
from keras.preprocessing import image
def frame_to_tensor(frm):
    img = cv2.resize(frm, (48,48), interpolation = cv2.INTER_AREA)
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x = image.img_to_array(gray_image)
    #res = np.concatenate([x[np.newaxis]])
    return np.expand_dims(x, axis=0)


In [ ]:
strEmotion = ['Angry','Disgust','Fear','Happy','Sad','Surprise','Neutral']
def get_emotion_frame(frm):
    emotion_class = model_saved.predict_classes(frame_to_tensor(frm))
    emotion = str(strEmotion[np.asscalar(emotion_class)])
    return emotion

In [ ]:
from cv2 import __version__
__version__

In [ ]:
import cv2
import numpy as np
 
# Create a VideoCapture object and read from input file
# If the input is the camera, pass 0 instead of the video file name
#cap = cv2.VideoCapture(0)
filepath = 'C:/Work/1. Coursera/4. MLND/machine-learning-master/machine-learning-master/projects/dog-project-master/to/emoticon/'
cap = cv2.VideoCapture('Test.mp4')

cascade = cv2.CascadeClassifier('C:/Work/1. Coursera/4. MLND/machine-learning-master/machine-learning-master/projects/dog-project-master/to/emoticon/haarcascades/haarcascade_frontalface_alt.xml')
 
# Check if camera opened successfully
if (cap.isOpened()== False): 
  print("Error opening video stream or file")
 
# Read until video is completed
while(cap.isOpened()):
  # Capture frame-by-frame
  ret, frame = cap.read()
  if ret == True:
 
    #find faces
    grayImage = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    body = cascade.detectMultiScale(
        grayImage,
        scaleFactor = 1.5,
        minNeighbors = 5,
        minSize = (30,30),
        flags = cv2.CASCADE_SCALE_IMAGE
    )
    
    emo = get_emotion_frame(frame)
    
    for (x,y,w,h) in body:
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),4)
        font = cv2.FONT_HERSHEY_SIMPLEX
        cv2.putText(frame,emo,(x,y), font, .5,(255,255,255),2,cv2.LINE_AA)
        #roi_gray = grayImage[y:y+h, x:x+w]
        #roi_color = frame[y:y+h, x:x+w]
    
    # Display the resulting frame
    cv2.imshow('Frame',frame)
 
    # Press Q on keyboard to  exit
    if cv2.waitKey(25) & 0xFF == ord('q'):
      break
 
  # Break the loop
  else: 
    break
 
# When everything done, release the video capture object
cap.release()
 
# Closes all the frames
cv2.destroyAllWindows()

In [ ]:
#from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib
import cv2

In [ ]:
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
face_detector = dlib.get_frontal_face_detector()
def land2coords(landmarks, dtype="int"):
    # initialize the list of tuples
    # (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
 
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (a, b)-coordinates
    for i in range(0, 68):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)
 
    # return the list of (a, b)-coordinates
    return coords

In [ ]:
    #import cv2
    #from imutils import face_utils
    import numpy as np
    import argparse
    import imutils
    import dlib
    import cv2
    filepath = 'C:/Work/1. Coursera/4. MLND/machine-learning-master/machine-learning-master/projects/dog-project-master/to/emoticon'
    vid = cv2.VideoCapture('test_vids/charlie.webm')
    #vid = cv2.VideoCapture(filepath + 'Test.mp4')
    #vid = cv2.VideoCapture(0)
 
    if (vid.isOpened()== False): 
      print("Error opening video stream or file")

    while vid.isOpened():
        _,frame = vid.read()
 
        # resizing frame
        # you can use cv2.resize but I recommend imutils because its easy to use
        frame = imutils.resize(frame, width=800)
 
        # grayscale conversion of image because it is computationally efficient
        # to perform operations on single channeled (grayscale) image
        frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
 
        # detecting faces
        face_boundaries = face_detector(frame_gray,0)
 
        for (enum,face) in enumerate(face_boundaries):
            # let's first draw a rectangle on the face portion of image
            x = face.left()
            y = face.top()
            w = face.right() - x
            h = face.bottom() - y
            # Drawing Rectangle on face part
            cv2.rectangle(frame, (x,y), (x+w, y+h), (120,160,230),2)
 
            # Now when we have our ROI(face area) let's
            # predict and draw landmarks
            landmarks = landmark_predictor(frame_gray, face)
            # converting co-ordinates to NumPy array
            landmarks = land2coords(landmarks)
            for (a,b) in landmarks:
                # Drawing points on face
                cv2.circle(frame, (a, b), 2, (255, 0, 0), -1)
 
            # Writing face number on image
            #cv2.putText(frame, "Face :{}".format(enum + 1), (x - 10, y - 10),
            #            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 128), 2)
            emoz = frame_to_emo(frame)
            cv2.putText(frame, "Face :" + str(emoz), (x - 10, y - 10),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 128), 2)
 
        cv2.imshow("frame", frame)
 
        #  Stop if 'q' is pressed
        if cv2.waitKey(1) == ord('q'):
            break;
    vid.release()
    cv2.destroyAllWindows()

In [ ]:
from keras.preprocessing import image                  
#img_path = 'test_images/mark.jpg'
img_path = result['paths'][109]
#img = image.load_img(img_path,grayscale=True)
img = cv2.imread(img_path)
img.shape
plt.imshow(img)
plt.show()
frame = imutils.resize(img, width=800)
     
        # grayscale conversion of image because it is computationally efficient
        # to perform operations on single channeled (grayscale) image
frame_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
 
        # detecting faces
face_boundaries = face_detector(frame_gray,0)
 
for (enum,face) in enumerate(face_boundaries):
            # let's first draw a rectangle on the face portion of image
    x = face.left()
    y = face.top()
    w = face.right() - x
    h = face.bottom() - y
            # Drawing Rectangle on face part
    cv2.rectangle(frame, (x,y), (x+w, y+h), (120,160,230),2)
 
            # Now when we have our ROI(face area) let's
            # predict and draw landmarks
    landmarks = landmark_predictor(frame_gray, face)
            # converting co-ordinates to NumPy array
    landmarks = land2coords(landmarks)
      

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import matplotlib.pyplot as plt
#print(scaler.fit_transform(landmarks))
#plt.plot([1,2,3,4])

#plt.plot(df_new['lmarks'][1])
#plt.show()

In [ ]:
plt.plot(scaler.fit_transform(df_new['lmarks'][1]))
plt.show()
scaler.fit_transform(df_new['lmarks'][1])

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.layers import Dropout, Flatten, Dense, LSTM
from keras.models import Sequential
from keras.initializers import TruncatedNormal,glorot_normal
import math


In [ ]:
from sklearn.datasets import load_files       
from keras.utils import np_utils
import numpy as np
from glob import glob

# define function to load train, test, and validation datasets
def load_dataset(path):
    data = load_files(path)
    emo_files = np.array(data['filenames'])
    emo_targets = np_utils.to_categorical(np.array(data['target']), 7)
    return emo_files, emo_targets

# load train, test, and validation datasets
try:
    train_files, train_targets = load_dataset('cohn-kanade')
except:
    pass
#valid_files, valid_targets = load_dataset('fer/PrivateTest')
#test_files, test_targets = load_dataset('fer/PublicTest')
#emo_names = [item[13:-1] for item in sorted(glob("cohn-kanade/S010/*/"))]

In [ ]:
print(len(train_files))

In [ ]:
data_paths = glob("cohn-kanade/*/*")

In [ ]:
print((data_paths[5]))

In [ ]:
import os
root = ("cohn-kanade")
filepaths = []
for path, subdirs, files in os.walk(root):
    for name in files:
        filepaths.append(os.path.join(path, name))
        #print (os.path.join(path, name))

In [ ]:
print((filepaths[0:10]))
len(filepaths)


In [ ]:

df_paths = pd.DataFrame.from_dict({'paths':filepaths})
df_paths

In [ ]:
df_paths = pd.DataFrame.from_dict({'paths':filepaths})
df_paths['key'] = df_paths.paths.str[:20]
emomap = pd.read_csv("Cohn-Kanade Database FACS codes_v2.1.csv")

In [ ]:
import os
root = ("cohn-kanade")
filepaths = []
for path, subdirs, files in os.walk(root):
    for name in files:
        filepaths.append(os.path.join(path, name))
        #print (os.path.join(path, name))
df_paths = pd.DataFrame.from_dict({'paths':filepaths})
df_paths['key'] = df_paths.paths.str[:20]
emomap = pd.read_csv("Cohn-Kanade Database FACS codes_v2.1.csv")
result = pd.merge(df_paths, emomap, on='key', how='inner')
#result['paths'][0]
#result['paths']
result.shape

In [ ]:
from keras.preprocessing import image                  
#img_path = 'test_images/mark.jpg'
def img_to_landmarks(imgpath):
    img = cv2.imread(imgpath)
    frame = imutils.resize(img, width=800)
    frame_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_boundaries = face_detector(frame_gray,0)
    for (enum,face) in enumerate(face_boundaries):
        x = face.left()
        y = face.top()
        w = face.right() - x
        h = face.bottom() - y
        #cv2.rectangle(frame, (x,y), (x+w, y+h), (120,160,230),2)
        landmarks = landmark_predictor(frame_gray, face)
        landmarks = land2coords(landmarks)
        return landmarks

In [ ]:
face_land = []
for path in df_paths['paths']:
    face_land.append(img_to_landmarks(path))

In [ ]:
len(result['paths'])

In [ ]:
df_landmarks = pd.DataFrame.from_dict({'lmarks':face_land})

In [ ]:
df_landmarks.to_pickle("face_marks.pkl")

In [ ]:
df_new = pd.read_pickle("face_marks.pkl")

In [ ]:
df_new['lmarks'].shape


##Join "result" & "df_new" to get emotions with facial landmarks

In [ ]:
df_new.__class__

In [ ]:
import pip  
pip.main(['install', 'imutils'])

In [ ]:
import skvideo.io
import skvideo.datasets
videodata = skvideo.io.vread(skvideo.datasets.bigbuckbunny())


In [ ]:
import sys
!conda install --yes --prefix {sys.prefix} sk-video

In [ ]:
df_new['lmarks'][0]
emomap = pd.read_csv("Cohn-Kanade Database FACS codes_v2.1.csv")
emomap['Emotion'].shape
df_new['lmarks'].shape[0]
scaler.fit_transform(df_new['lmarks'][i]).shape
df_new['lmarks'][i].shape

In [ ]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
#df_new['lmarks_scaled'] = []
y_emo =[]
for i in range(df_new['lmarks'].shape[0]):
    df_new['lmarks'][i]= scaler.fit_transform(df_new['lmarks'][i]) 

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor2(img_path):
    img = image.load_img(img_path,grayscale=True, target_size=(48, 48))
    x = image.img_to_array(img)
    x = x[:, :, 0]
    #x.transpose(2,0,1).reshape(3,-1)
    return np.expand_dims(x, axis=0)

def paths_to_tensor2(img_paths):
    list_of_tensors = [path_to_tensor2(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
#path_to_tensor2(result['paths'][0]).shape
img = image.load_img(result['paths'][0],grayscale=True, target_size=(48, 48))
x = image.img_to_array(img)
x = x[:, :, 0]
x.shape


In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

X_tensors = paths_to_tensor2(result['paths']).astype('float32')/255
#X_lstm = paths_to_tensor(result['paths']).astype('float32')/255

X_tensors.shape
#valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
#test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor2(img_path):
    img = image.load_img(img_path,grayscale=True, target_size=(48, 48))
    x = image.img_to_array(img)
    x = x[:, :, 0]
    #x.transpose(2,0,1).reshape(3,-1)
    return np.expand_dims(x, axis=0)

def paths_to_tensor2(img_paths):
    list_of_tensors = [path_to_tensor2(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
result['paths'].shape


In [ ]:
X_tensors[0].shape

In [ ]:
#y_catg = np_utils.to_categorical(np.array(result['Emotion']), 6)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

y_emo = result['Emotion'].groupby(result['seq']).tail(1).reset_index()

lb_emo = LabelEncoder()
y_emo_lstm = lb_emo.fit_transform(result['Emotion'])
y_emo_lstm = pd.DataFrame.from_dict({'emo_code':y_emo_lstm})
y_emo_codes = lb_emo.fit_transform(y_emo['Emotion'])
y_emo_codes = pd.DataFrame.from_dict({'emo_code':y_emo_codes})
ohe = OneHotEncoder()
#y_emo_codes
y_emo_oh = ohe.fit_transform(y_emo_codes)
y_emo_oh.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tensors,y_emo_lstm['emo_code'], test_size=0.33, random_state=42, shuffle = False)
X_train[1].shape


In [ ]:
X_tensors.shape

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM
from keras.optimizers import Adam



model= Sequential()
model.add(LSTM(128,input_shape = (48, 48) ,activation = 'relu', return_sequences = True))
model.add(Dropout(0.2))

model.add(LSTM(128,activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(32, activation = 'relu'))
model.add(Dropout(0.2))

model.add(Dense(6, activation = 'softmax'))

model.summary()

#opt = optimizers.Adam(lr = 1e-3, decay = 1e-5)

model.compile(loss='sparse_categorical_crossentropy', optimizer = "Adam", metrics = ['accuracy'])
model.fit(X_train,y_train, epochs = 10, validation_data = (X_test,y_test))




In [ ]:
import os
import pandas as pd
root = ("cohn-kanade")
filepaths = []
folders = []
seq_length = 10
for path, subdirs, files in os.walk(root):
    for name in files:
        filepaths.append(os.path.join(path, name))
        folders.append(path)
        #print (os.path.join(path, name))
df_paths = pd.DataFrame.from_dict({'paths':filepaths,'seq':folders})
df_paths['key'] = df_paths.paths.str[:20]
#df_paths
emomap = pd.read_csv("Cohn-Kanade Database FACS codes_v2.1.csv")
result = pd.merge(df_paths, emomap, on='key', how='inner')
#result['paths'][0]
#result['paths']
result2 = result.groupby('seq').tail(seq_length)
seq_10 = result2.groupby('seq').size().reset_index(name='counts')
seq_10 = seq_10.loc[seq_10['counts']==seq_length]

result = pd.merge(result2, seq_10, on='seq', how='inner')
result.shape
#result

In [ ]:
face_land = []
for path in df_paths['paths']:
    face_land.append(img_to_landmarks(path))

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

#X_tensors = paths_to_tensor2(result['paths']).astype('float32')/255
X_lstm = paths_to_tensor(result['paths']).astype('float32')/255
X_lstm = X_lstm.reshape(451,5,48,48,1)
X_lstm.shape
#valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
#test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
y_emo = result['Emotion'].groupby(result['seq']).tail(1).reset_index()

lb_emo = LabelEncoder()

y_emo_codes = lb_emo.fit_transform(y_emo['Emotion'])
y_emo_codes = pd.DataFrame.from_dict({'emo_code':y_emo_codes})
ohe = OneHotEncoder()
#y_emo_codes
y_emo_oh = ohe.fit_transform(y_emo_codes).toarray()
y_emo_codes
#y_emo_oh = np_utils.to_categorical(np.array(y_emo_codes), 6)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_lstm,y_emo_codes['emo_code'], test_size=0.3, random_state=42, shuffle = True)
y_train[1]


In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, ConvLSTM2D, BatchNormalization, Conv2D, Flatten
from keras.optimizers import Adam

model = Sequential()
model.add(ConvLSTM2D(filters=10, kernel_size=(3,3), activation='tanh',
                             input_shape=[5] + list((48,48)) + [1],
                             data_format='channels_last', return_sequences=True))
model.add(BatchNormalization())
model.add(ConvLSTM2D(filters=10, kernel_size=(3,3), activation='tanh',
                             input_shape=(5, 1) + (48,48,1),
                             data_format='channels_last', return_sequences=True))
model.add(BatchNormalization())

model.add(Dropout(0.2))

model.add(ConvLSTM2D(filters=10, kernel_size=(3,3), activation='tanh'))
model.add(BatchNormalization())

#model.add(ConvLSTM2D(filters=10, kernel_size=(4,4), activation='sigmoid'))
#model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(filters=4, kernel_size=(3,3), activation="relu", data_format="channels_last"))
model.add(Flatten())
#model.add(Dense(units=100, activation="relu"))
model.add(Dropout(0.2))
model.add(Dense(6,activation='softmax',name="Output"))

model.summary()

model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#model.fit(X_train,y_train, batch_size=10, epochs=5, validation_data=(X_test,y_test))
                       #callbacks=[ReduceLROnPlateau(), EarlyStopping(patience=3)])

    
from keras.callbacks import ModelCheckpoint,CSVLogger
epochs = 10

csv_logger = CSVLogger('logs/training.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.ConvLSTM2D.hdf5', 
                               verbose=1, save_best_only=True)

hist = model.fit(X_train,y_train, 
          validation_data=(X_test,y_test),
          epochs=epochs, batch_size=10, callbacks=[checkpointer,csv_logger], verbose=0)

In [ ]:
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, ConvLSTM2D, BatchNormalization, Conv2D, Flatten
from keras.optimizers import Adam



# Sequential flows in parallel - cnn+face landmarks

In [1]:
import pandas as pd
import numpy as np
import os
root = ("cohn-kanade")
filepaths = []
folders = []
seq_length = 10
for path, subdirs, files in os.walk(root):
    for name in files:
        filepaths.append(os.path.join(path, name))
        folders.append(path)
        #print (os.path.join(path, name))
df_paths = pd.DataFrame.from_dict({'paths':filepaths,'seq':folders})
df_paths['key'] = df_paths.paths.str[:20]
#df_paths
emomap = pd.read_csv("Cohn-Kanade Database FACS codes_v2.2.csv")
result = pd.merge(df_paths, emomap, on='key', how='inner')
#result['paths'][0]
#result['paths']
result2 = result.groupby('seq').tail(seq_length)
seq_10 = result2.groupby('seq').size().reset_index(name='counts')
seq_10 = seq_10.loc[seq_10['counts']==seq_length]

#result = pd.merge(result2, seq_10, on='seq', how='inner')
result.shape
#result

(7334, 7)

In [ ]:
#df_paths

In [2]:
from keras.preprocessing import image                  
#img_path = 'test_images/mark.jpg'
def img_to_landmarks(imgpath):
    img = cv2.imread(imgpath)
    frame = imutils.resize(img, width=800)
    frame_gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    face_boundaries = face_detector(frame_gray,0)
    for (enum,face) in enumerate(face_boundaries):
        x = face.left()
        y = face.top()
        w = face.right() - x
        h = face.bottom() - y
        #cv2.rectangle(frame, (x,y), (x+w, y+h), (120,160,230),2)
        landmarks = landmark_predictor(frame_gray, face)
        landmarks = land2coords(landmarks)
        return landmarks

Using TensorFlow backend.


In [3]:
import dlib
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
face_detector = dlib.get_frontal_face_detector()
def land2coords(landmarks, dtype="int"):
    # initialize the list of tuples
    # (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
 
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (a, b)-coordinates
    for i in range(0, 68):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)
 
    # return the list of (a, b)-coordinates
    return coords

In [4]:
import cv2                
import matplotlib.pyplot as plt                        
#from imutils import face_utils
import numpy as np
import argparse
import imutils
import dlib



In [ ]:
face_land = []
for path in result['paths']:
    face_land.append(img_to_landmarks(path))

In [ ]:
df_landmarks = pd.DataFrame.from_dict({'lmarks':face_land})
df_landmarks.to_pickle("face_landmarks_marks.pkl")

In [ ]:
df_landmarks = pd.read_pickle("face_landmarks_marks.pkl")

In [ ]:
#face_land = np.array(face_land)
#df_landmarks['lmarks'].shape
x_lmark = np.vstack(df_landmarks['lmarks'])
x_lmark = np.reshape(x_lmark,(7334,68,2))
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
x_lmark = np.array([scaler.fit_transform(x) for x in x_lmark[:]])
#df_landmarks['lmarks'][0]
#x_lmark

In [ ]:
df_landmarks_fer_train['lmarks'].shape

In [ ]:
from keras.preprocessing import image                  
from tqdm import tqdm

def path_to_tensor(img_path):
    img = image.load_img(img_path,grayscale=True, target_size=(48, 48))
    x = image.img_to_array(img)
    #x = x[:, :, 0]
    #x.transpose(2,0,1).reshape(3,-1)
    return np.expand_dims(x, axis=0)

def paths_to_tensor(img_paths):
    list_of_tensors = [path_to_tensor(img_path) for img_path in tqdm(img_paths)]
    return np.vstack(list_of_tensors)

In [ ]:
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True                 

X_tensors = paths_to_tensor(result['paths']).astype('float32')/255
#X_lstm = paths_to_tensor(result['paths']).astype('float32')/255

X_tensors.shape
#valid_tensors = paths_to_tensor(valid_files).astype('float32')/255
#test_tensors = paths_to_tensor(test_files).astype('float32')/255

In [ ]:
result['Emotion'].shape

In [35]:
#y_catg = np_utils.to_categorical(np.array(result['Emotion']), 6)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from keras.utils import np_utils
#y_emo = result['Emotion'].groupby(result['seq']).tail(1).reset_index()

lb_emo = LabelEncoder()
y_emo_lstm = lb_emo.fit_transform(result['Emotion'])
y_emo_lstm = pd.DataFrame.from_dict({'emo_code':y_emo_lstm})
#y_emo_codes = lb_emo.fit_transform(y_emo_lstm['emo_code'])
#y_emo_codes = pd.DataFrame.from_dict({'emo_code':y_emo_codes})
ohe = OneHotEncoder()
#y_emo_codes
y_emo_oh = ohe.fit_transform(y_emo_lstm)

emo_targets = np_utils.to_categorical(np.array(y_emo_lstm['emo_code']), 7)
lb_emo.get_params(result['Emotion'])
list(lb_emo.classes_)

['Anger', 'Disgust', 'Fear', 'Happy', 'Neutral', 'Sad', 'Surprise']

In [ ]:
emo_targets

In [ ]:
from sklearn.model_selection import train_test_split
X1_train, X1_test, y_train, y_test = train_test_split(X_tensors,emo_targets, test_size=0.33, random_state=42, shuffle = True)
X2_train, X2_test, y_train, y_test = train_test_split(x_lmark,emo_targets, test_size=0.33, random_state=42, shuffle = True)
X2_train[1].shape


In [ ]:
x_lmark.shape

In [ ]:
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D,GlobalMaxPooling2D
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, Merge
from keras.models import Sequential
from keras.initializers import TruncatedNormal,glorot_normal
import math

model = Sequential()

n = 0.01
model.add(Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu',input_shape=(48, 48, 1)
                 ,kernel_initializer=glorot_normal(seed=0)
                 ,bias_initializer='zeros',name = "Input"))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=3, strides=2))

n = 0.01
model.add(Conv2D(filters=64, kernel_size=5, strides=1, padding='same', activation='relu'
                 ,kernel_initializer=glorot_normal(seed=0)
                 ,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(MaxPooling2D(pool_size=3, strides=2))

model.add(Conv2D(filters=128, kernel_size=4, strides=1, padding='same', activation='relu'
                 ,kernel_initializer=glorot_normal(seed=0)
                 ,bias_initializer='zeros'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
#model.add(MaxPooling2D(pool_size=2, strides=2))


#model.add(Flatten())

model.add(Dense(1024,activation = 'relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(40,activation = 'relu'))
model.add(Dropout(0.2))
#model.add(Dense(15,activation = 'relu'))
#n = math.sqrt(2.0/(9*256))
model.add(Flatten())

model_lmark = Sequential()
model_lmark.add(Dense(1024,activation = 'relu',input_shape=(68,2)))
model.add(Dropout(0.2))
model_lmark.add(Dense(40,activation = 'relu'))
model_lmark.add(Flatten())



merged = Sequential()
merged.add(Merge([model, model_lmark], mode= 'concat'))
model.add(Dropout(0.2))

merged.add(Dense(7,activation='softmax',name="Output"))

merged.summary()
merged.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])
#model.compile(optimizer="Adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
#model.fit([X_tensors,x_lmark],y_emo_lstm['emo_code'])



In [ ]:
#merged.fit([X_tensors,x_lmark],y_emo_lstm['emo_code'])
#merged.fit([X_tensors,x_lmark],emo_targets)

In [ ]:
#merged.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


from keras.callbacks import ModelCheckpoint,CSVLogger
epochs = 30

csv_logger = CSVLogger('logs/training_merged.csv')
checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.merged.hdf5', 
                               verbose=1, save_best_only=True)

hist = merged.fit([X1_train,X2_train],y_train, 
          validation_data=([X1_test,X2_test],y_test),
          epochs=epochs, batch_size=100, callbacks=[checkpointer,csv_logger], verbose=0)


In [ ]:
merged.load_weights('saved_models/weights.best.merged.hdf5')
#merged.save('saved_models/merged_best.h5')

In [ ]:
# Save the weights
merged.save_weights('saved_models/merged_weights.h5')

# Save the model architecture
with open('saved_models/merged_architecture.json', 'w') as f:
    f.write(merged.to_json())

In [4]:
from keras.models import model_from_json

# Model reconstruction from JSON file
with open('saved_models/merged_architecture.json', 'r') as f:
    modelx = model_from_json(f.read())

# Load weights into the new model
modelx.load_weights('saved_models/merged_weights.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


C:\Anaconda\42\envs\emo\lib\site-packages\keras\engine\topology.py:1206: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  return cls(**config)


In [5]:
import dlib
landmark_predictor = dlib.shape_predictor('shape_predictor_68_face_landmarks.dat')
face_detector = dlib.get_frontal_face_detector()
def land2coords(landmarks, dtype="int"):
    # initialize the list of tuples
    # (x, y)-coordinates
    coords = np.zeros((68, 2), dtype=dtype)
 
    # loop over the 68 facial landmarks and convert them
    # to a 2-tuple of (a, b)-coordinates
    for i in range(0, 68):
        coords[i] = (landmarks.part(i).x, landmarks.part(i).y)
 
    # return the list of (a, b)-coordinates
    return coords

In [ ]:
test_image = 'test_images\Alan_Greenspan_0003.jpg'

test_tensor = path_to_tensor(test_image).astype('float32')/255
test_lmark = img_to_landmarks(test_image)
test_lmark = scaler.fit_transform(test_lmark)
#result['paths']
test_lmark = np.expand_dims(test_lmark, axis=0)
test_lmark.shape
modelx.predict([test_tensor,test_lmark])
list(lb_emo.classes_)[np.argmax(modelx.predict([test_tensor,test_lmark]))]

In [6]:
from keras.preprocessing import image
def face_to_emo(frm):
    img = cv2.resize(frm, (48,48), interpolation = cv2.INTER_AREA)
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    x = image.img_to_array(gray_image)
    #res = np.concatenate([x[np.newaxis]])
    x1 = np.expand_dims(x, axis=0)
    
    x2 = face_to_landmarks(frm)
    x2 = scaler.fit_transform(x2)
    x2 = np.expand_dims(x2, axis=0)
    return list(lb_emo.classes_)[np.argmax(modelx.predict([x1,x2]))]


In [7]:
from keras.preprocessing import image                  
#img_path = 'test_images/mark.jpg'
def face_to_landmarks(frm):
    #img = cv2.imread(imgpath)
    #frm = imutils.resize(frm, width=800)
    frame_gray = cv2.cvtColor(frm,cv2.COLOR_BGR2GRAY)
    face_boundaries = face_detector(frame_gray,0)
    for (enum,face) in enumerate(face_boundaries):
        x = face.left()
        y = face.top()
        w = face.right() - x
        h = face.bottom() - y
        #cv2.rectangle(frame, (x,y), (x+w, y+h), (120,160,230),2)
        landmarks = landmark_predictor(frame_gray, face)
        landmarks = land2coords(landmarks)
        return landmarks

In [ ]:
Alan_Greenspan_0003

In [8]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
import matplotlib.pyplot as plt
#print(scaler.fit_transform(landmarks))
#plt.plot([1,2,3,4])

#plt.plot(df_new['lmarks'][1])
#plt.show()

In [9]:
#y_catg = np_utils.to_categorical(np.array(result['Emotion']), 6)
from sklearn.preprocessing import LabelEncoder, OneHotEncoder

y_emo = result['Emotion'].groupby(result['seq']).tail(1).reset_index()

lb_emo = LabelEncoder()
y_emo_lstm = lb_emo.fit_transform(result['Emotion'])
y_emo_lstm = pd.DataFrame.from_dict({'emo_code':y_emo_lstm})
y_emo_codes = lb_emo.fit_transform(y_emo['Emotion'])
y_emo_codes = pd.DataFrame.from_dict({'emo_code':y_emo_codes})
ohe = OneHotEncoder()
#y_emo_codes
y_emo_oh = ohe.fit_transform(y_emo_codes)
y_emo_oh.shape

(486, 7)

In [12]:
    #import cv2
    #from imutils import face_utils
    import numpy as np
    import argparse
    import imutils
    import dlib
    import cv2
    filepath = 'C:/Work/1. Coursera/4. MLND/machine-learning-master/machine-learning-master/projects/dog-project-master/to/emoticon'
    vid = cv2.VideoCapture('test_vids/EdwardSnowden.mp4')
    #vid = cv2.VideoCapture(filepath + 'Test.mp4')
    #vid = cv2.VideoCapture(0)
    fourcc = cv2.VideoWriter_fourcc(*'MJPG')
    #out = cv2.VideoWriter('output.avi', fourcc, 20.0, (600,800))
    count = 0
    res_probs = []
    res_face = []
    frm_count = []
            
    if (vid.isOpened()== False): 
      print("Error opening video stream or file")

    while vid.isOpened():
        ret,frame = vid.read()
        if ret == True:
            # resizing frame
            # you can use cv2.resize but I recommend imutils because its easy to use
            frame = imutils.resize(frame, width=800)

            # grayscale conversion of image because it is computationally efficient
            # to perform operations on single channeled (grayscale) image
            frame_gray = cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)

            # detecting faces
            face_boundaries = face_detector(frame_gray,0)
            for (enum,face) in enumerate(face_boundaries):
                # let's first draw a rectangle on the face portion of image
                x = face.left()
                y = face.top()
                w = face.right() - x
                h = face.bottom() - y
                # Drawing Rectangle on face part
                #cv2.rectangle(frame, (x,y), (x+w, y+h), (120,160,230),2)

                # Now when we have our ROI(face area) let's
                # predict and draw landmarks
                landmarks = landmark_predictor(frame_gray, face)
                # converting co-ordinates to NumPy array
                landmarks = land2coords(landmarks)
                x2 = np.expand_dims(scaler.fit_transform(landmarks), axis=0)

                face = cv2.resize(frame, (48,48), interpolation = cv2.INTER_AREA)
                gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
                x1 = image.img_to_array(gray_face)
                x1 = np.expand_dims(x1, axis=0)

                for (a,b) in landmarks:
                    # Drawing points on face
                    cv2.circle(frame, (a, b), 2, (255, 0, 0), -1)
                try:
                    res_probs.append(modelx.predict([x1,x2])[0])
                    #print(modelx.predict([x1,x2])[0])
                    res_face.append(enum)
                    frm_count.append(count)
                    emoz = list(lb_emo.classes_)[np.argmax(modelx.predict([x1,x2]))] #face_to_emo(frame[y:y+h, x:x+w])
                    max1 = max(modelx.predict([x1,x2]))
                    cv2.rectangle(frame, (x,y), (x+w, y+h), (120,160,230),2)
                    #print(np.argmax(modelx.predict([x1,x2])))
                    cv2.putText(frame, "Feeling :" + str(emoz), (x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 128), 2)
                    #cv2.putText(frame, "Feeling :" + str(max1), (20, 20+enum*20),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 128), 2)
                except ValueError:
                    cv2.putText(frame, "Scanning.." , (x - 10, y - 10),cv2.FONT_HERSHEY_SIMPLEX, 0.5, (200, 200, 128), 2)
                else:
                    pass

            #out.write(frame)
            cv2.imwrite("out_frames/frame%d.jpg" % count, frame)
            cv2.imshow("frame", frame)

            #print(frame.shape)
            #  Stop if 'q' is pressed
            if cv2.waitKey(1) == ord('q'):
                break;
            count += 1
    vid.release()
    
    #out.release()
    
    cv2.destroyAllWindows()
    

C:\Anaconda\42\envs\emo\lib\site-packages\sklearn\utils\validation.py:444: DataConversionWarning: Data with input dtype int32 was converted to float64 by MinMaxScaler.
  warnings.warn(msg, DataConversionWarning)


In [13]:
#emo_df = pd.DataFrame(frm_count,res_face,res_probs, columns=['Frame','face','probs'])
#print(res_probs)
df = pd.DataFrame(res_probs,columns=list(lb_emo.classes_))
df['Frames'] = frm_count
df['Face'] = res_face
df.to_csv('Snowden_Interview.csv')